<a href="https://colab.research.google.com/github/Dhyanesh-Panchal/GSR-research-and-stress-detection/blob/master/notebooks/LSTM_GSR_Phasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm.notebook import tqdm as bar
import numpy as np

In [39]:
data = pd.read_csv("https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_1.csv").drop("Unnamed: 0",axis=1)
data = data[data["video"].isin([5,6,7,8])]

In [40]:
for id in bar(range(2,31)):
    sub_data = pd.read_csv(f"https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_{id}.csv").drop("Unnamed: 0",axis=1)
    sub_data = sub_data[sub_data["video"].isin([5,6,7,8])]
    data = pd.concat([data,sub_data],axis=0)

  0%|          | 0/29 [00:00<?, ?it/s]

In [44]:
data = data.reset_index(drop=True)

## Approach_1

In [21]:
def generate_windows_df(df: pd.DataFrame, window_size: int, window_gap: int, column_name: str):
        def rolling_window(df: pd.DataFrame, window_size: int, window_gap: int):
            windows = []
            for i in bar(range(0, len(df)-window_size+1, window_gap), desc="generating window"):
                single_window = df.iloc[i:i+window_size]
                # print(single_window)
                windows.append(single_window)
            return windows

        if window_size < len(df):
            windowed_data = rolling_window(
                df, window_size, window_gap)
            array = []
            for frame in bar(windowed_data, desc="preparing dataframe"):
                array.append(list(frame[column_name]))

            window_df = pd.DataFrame(np.array(array))
        else:
            return df
        return window_df

def pre_process(df):
  phasic_windowed = generate_windows_df(df, window_size=100, window_gap=1, column_name="EDA_Phasic")

  X = np.array(phasic_windowed).reshape(len(phasic_windowed),100,1)
  # inserted the initial arousal value during the "start" of window
  # X = np.concatenate((X,np.array(df['class_2_arousal'][99:]).reshape(len(df['class_2_arousal']) - 99,1,1)) , axis=1)

  y = np.array(df["class_2_arousal"])[99:]

  return X,y

In [22]:
def prepare_data(subject_ID):
  data_df = pd.read_csv(f"https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_{subject_ID}.csv").drop("Unnamed: 0",axis=1)
  data_df = data_df[data_df["video"].isin([5,6,7,8])]
  filtered_data = data_df[(data_df["arousal"]!=5) & (data_df["valence"]!=5)]
  filtered_data = filtered_data.reset_index().drop("index",axis=1)
  X,y = pre_process(filtered_data)
  y = pd.get_dummies(y)
  return X,y

In [23]:
X,y = prepare_data(1)

generating window:   0%|          | 0/11394 [00:00<?, ?it/s]

preparing dataframe:   0%|          | 0/11394 [00:00<?, ?it/s]

# LSTM model

In [5]:
import tensorflow as tf
from tensorflow import keras

In [36]:
def build_model(input_shape):
    model = keras.Sequential()

    # LSTM layers
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))

    # Dense Layer
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    # Dense Layer
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # Output layer
    model.add(keras.layers.Dense(2, activation="softmax"))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [37]:
lstm_model = build_model(X.shape[1:])

In [38]:
lstm_model.fit(X,y, epochs=15, validation_split=0.2)

Epoch 1/15
291/291 [==============================] - 9s 17ms/step - loss: 0.4780 - accuracy: 0.7970 - val_loss: 0.6953 - val_accuracy: 0.5178
Epoch 2/15
291/291 [==============================] - 4s 14ms/step - loss: 0.4351 - accuracy: 0.8198 - val_loss: 0.8509 - val_accuracy: 0.5247
Epoch 3/15
291/291 [==============================] - 4s 15ms/step - loss: 0.4533 - accuracy: 0.8084 - val_loss: 0.7683 - val_accuracy: 0.4383
Epoch 4/15
291/291 [==============================] - 4s 13ms/step - loss: 0.4412 - accuracy: 0.8025 - val_loss: 0.6187 - val_accuracy: 0.5342
Epoch 5/15
291/291 [==============================] - 4s 13ms/step - loss: 0.4348 - accuracy: 0.8128 - val_loss: 0.7236 - val_accuracy: 0.6151
Epoch 6/15
291/291 [==============================] - 5s 16ms/step - loss: 0.4771 - accuracy: 0.7917 - val_loss: 0.7809 - val_accuracy: 0.5187
Epoch 7/15
291/291 [==============================] - 5s 16ms/step - loss: 0.4401 - accuracy: 0.8096 - val_loss: 0.7496 - val_accuracy: 0.5200

In [26]:
training_metrices = []

for subject_no in range(1,26):
  print(f"\n\n\t\t********** SUBJECT - {subject_no} **********\n\n")
  X,y = prepare_data(subject_ID=subject_no)

  '''
  Training 10 epochs for each subject data with validation split of 20% on same Data
  '''
  fitting = lstm_model.fit(X,y,epochs=5,validation_split=0.2)
  training_metrices.append(fitting)



		********** SUBJECT - 1 **********




generating window:   0%|          | 0/11394 [00:00<?, ?it/s]

preparing dataframe:   0%|          | 0/11394 [00:00<?, ?it/s]

Epoch 1/5
285/285 [==============================] - 5s 16ms/step - loss: 0.3159 - accuracy: 0.9060 - val_loss: 0.0889 - val_accuracy: 1.0000
Epoch 2/5
285/285 [==============================] - 3s 12ms/step - loss: 0.3152 - accuracy: 0.9060 - val_loss: 0.1294 - val_accuracy: 1.0000
Epoch 3/5
285/285 [==============================] - 3s 12ms/step - loss: 0.3141 - accuracy: 0.9060 - val_loss: 0.1055 - val_accuracy: 1.0000
Epoch 4/5
285/285 [==============================] - 4s 15ms/step - loss: 0.3143 - accuracy: 0.9060 - val_loss: 0.1053 - val_accuracy: 1.0000
Epoch 5/5
285/285 [==============================] - 4s 13ms/step - loss: 0.3157 - accuracy: 0.9060 - val_loss: 0.1035 - val_accuracy: 1.0000


		********** SUBJECT - 2 **********




generating window:   0%|          | 0/9695 [00:00<?, ?it/s]

...

preparing dataframe:   0%|          | 0/10053 [00:00<?, ?it/s]

Epoch 1/5
252/252 [==============================] - 3s 13ms/step - loss: 0.5118 - accuracy: 0.7447 - val_loss: 0.2112 - val_accuracy: 1.0000
Epoch 2/5
252/252 [==============================] - 3s 12ms/step - loss: 0.5080 - accuracy: 0.7525 - val_loss: 0.2640 - val_accuracy: 1.0000
Epoch 3/5
252/252 [==============================] - 3s 12ms/step - loss: 0.4932 - accuracy: 0.7530 - val_loss: 0.2118 - val_accuracy: 1.0000
Epoch 4/5
252/252 [==============================] - 4s 16ms/step - loss: 0.4899 - accuracy: 0.7530 - val_loss: 0.1681 - val_accuracy: 1.0000
Epoch 5/5
252/252 [==============================] - 3s 12ms/step - loss: 0.4915 - accuracy: 0.7530 - val_loss: 0.2343 - val_accuracy: 1.0000


		********** SUBJECT - 25 **********




generating window:   0%|          | 0/11622 [00:00<?, ?it/s]

preparing dataframe:   0%|          | 0/11622 [00:00<?, ?it/s]

Epoch 1/5
291/291 [==============================] - 4s 14ms/step - loss: 0.5002 - accuracy: 0.7969 - val_loss: 0.4989 - val_accuracy: 0.5510
Epoch 2/5
291/291 [==============================] - 4s 12ms/step - loss: 0.4332 - accuracy: 0.8271 - val_loss: 0.5482 - val_accuracy: 0.5561
Epoch 3/5
291/291 [==============================] - 4s 12ms/step - loss: 0.4055 - accuracy: 0.8306 - val_loss: 0.6064 - val_accuracy: 0.5978
Epoch 4/5
291/291 [==============================] - 4s 14ms/step - loss: 0.3865 - accuracy: 0.8332 - val_loss: 0.6155 - val_accuracy: 0.6013
Epoch 5/5
291/291 [==============================] - 4s 12ms/step - loss: 0.3621 - accuracy: 0.8476 - val_loss: 0.5182 - val_accuracy: 0.6615


In [ ]:
test_df =